In [1]:
import requests
from geopy.geocoders import Nominatim
import time
import sqlite3 as sql

In [2]:
geolocator = Nominatim(user_agent="test")


conn = sql.connect("data.db")
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS locations(id integer PRIMARY KEY, country text)")
conn.commit()



while True:
    url = 'https://api.wheretheiss.at/v1/satellites/25544'
    headers = {'Accepts': 'application/json'}

    r = requests.get(url, headers=headers)
    new = r.json()
    
    location = geolocator.reverse(f"{new['latitude']}, {new['longitude']}")

    if location is None:
        print("location unknown (must be a body of water), going to sleep...")
        time.sleep(30)
        continue 

    else:
        print(location.address)
        cursor.execute("INSERT INTO locations(country) VALUES(?)", (location.address,))
        conn.commit()
        break

location unknown (must be a body of water), going to sleep...
location unknown (must be a body of water), going to sleep...


In [ ]:
cursor.execute("SELECT * FROM locations")
cursor.fetchall()

In [3]:
import pandas as pd

In [6]:
cursor.execute("INSERT INTO locations(country) VALUES ('Japan')")
conn.commit()

In [7]:
df = pd.read_sql(sql="SELECT * FROM locations", con=conn)
df

,id,country
0,1,France
1,2,United Kingdom
2,3,Japan


In [ ]:
df.to_csv("locs.csv", index=False)